## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server (to see things in real time)
- backend store: sqlite database (the metadata is stored here, metrics, tags, and so on...)
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root ./artifacts_local`
this generates a backend.db for the server in the current directory, and a folder called artifacts_local in the current directory.

*Note is different to:
mlflow ui --backend-store-uri sqlite:///backend.db.
mlflow.set_tracking_uri("sqlite:///mlflow.db")

in ths case, what we have seen so far is, the artifacts are stored in mlflow.db and also the metadata. We could not see the arrtifacts in the local system like now.

In [1]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='/home/yezer/projects/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/0', creation_time=1747997063211, experiment_id='0', last_update_time=1747997063211, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from mlflow.models.signature import infer_signature

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    # Define an input example
    input_example = X[:1]
    
    # Infer the signature
    signature = infer_signature(X, y)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="models",
        signature=signature,
        input_example=input_example
    )
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/05/23 11:56:40 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


default artifacts URI: '/home/yezer/projects/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/1/948a5971468a403a899ae3abcddb280b/artifacts'
🏃 View run adventurous-yak-260 at: http://127.0.0.1:5000/#/experiments/1/runs/948a5971468a403a899ae3abcddb280b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='/home/yezer/projects/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/1', creation_time=1747997291859, experiment_id='1', last_update_time=1747997291859, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='/home/yezer/projects/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/0', creation_time=1747997063211, experiment_id='0', last_update_time=1747997063211, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [7]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000")

In [8]:
client.search_registered_models()

[]

In [9]:
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/05/23 11:59:19 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1747997959030, current_stage='None', description='', last_updated_timestamp=1747997959030, name='iris-classifier', run_id='948a5971468a403a899ae3abcddb280b', run_link='', source='/home/yezer/projects/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/1/948a5971468a403a899ae3abcddb280b/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='1'>